# Check Events

Check event rankings from GP3S.

Copyright 2022 Michael George (AKA Logiqx).

This file is part of GP3S Query and is distributed under the terms of the GNU General Public License.

GP3S Query is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

GP3S Query is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with GP3S Query. If not, see https://www.gnu.org/licenses/.

## Import Common Modules

In [1]:
import os
import sys

import json

## Event IDs

Obtained from gps-speedsurfing.com

In [2]:
events = {}

users = {}

## Load Events

In [3]:
def getUid(sessionUrl):
    """Get user ID from session URL"""

    for param in sessionUrl.split('&'):
        if '=' in param:
            parts = param.split('=')
            if parts[0] == 'uid':
                uid = int(parts[1])
            
    return uid

In [4]:
def loadEventRankings(eventDir):
    """Load event rankings"""
    
    rankings = {}

    for rankingsFile in sorted(os.listdir(eventDir)):
        filename = os.path.join(eventDir, rankingsFile)
        with open(filename) as f:
            speedType = os.path.splitext(rankingsFile)[0]
            rankings[speedType] = json.load(f)

    return rankings

In [5]:
def parseEventRankings(rankings, verbose=False):
    """Parse event rankings"""

    bests = {}
    duplicates = {}
    
    for speedType, rows in rankings.items():
        if speedType not in bests:
            bests[speedType] = {}

        if speedType not in duplicates:
            duplicates[speedType] = {}
            
        for row in rows:
            uid = getUid(row['sessionurl'])
            
            if uid not in users:
                users[uid] = row['username']

            if uid in bests[speedType]:
                duplicates[speedType][uid] = row['speed']

            bests[speedType][uid] = row['speed']

    return bests, duplicates

In [6]:
def loadEventSessions(eventId):
    """Load event sessions"""
    
    sessions = {}

    filename = os.path.join(projdir, 'cache', 'EventSessions', str(eventId) + '.json')
    with open(filename) as f:
        sessions = json.load(f)

    for session in sessions:
        if eventId not in events:
            events[eventId] = session['eventname']
            break

    return sessions

In [7]:
def parseEventSessions(sessions):
    """Parse event sessions"""

    bests = {}
    
    for session in sessions:
        uid = getUid(session['sessionurl'])
                
        if uid not in users:
            users[uid] = session['username']

        for speedType in session:
            if speedType.startswith('speed_'):
                if speedType not in bests:
                    bests[speedType] = {}

                if uid not in bests[speedType]:
                    bests[speedType][uid] = 0
                    
                if session[speedType] > bests[speedType][uid]:
                    bests[speedType][uid] = session[speedType]

    return bests

In [8]:
def compareBests(eventId, rankingBests, rankingDuplicates, sessionBests, verbose=False):
    """Compare rankings against sessions"""

    rebuild = {}

    for speedType, uids in rankingDuplicates.items():
        for uid, speed in uids.items():
            if verbose:
                print('{} ({}) duplicate {} - {:.2f}'.format(users[uid], uid, speedType, speed))
            if uid not in rebuild:
                rebuild[uid] = set()
            rebuild[uid].add(speedType)
                
    for speedType, uids in rankingBests.items():
        if speedType not in sessionBests:
            if verbose:
                print('Warning: {} not in EventSessions'.format(speedType))
        else:
            for uid, speed in uids.items():
                if uid not in sessionBests[speedType]:
                    if verbose:
                        print('{} ({}) {} not in EventSessions'.format(users[uid], uid, speedType))
                    if uid not in rebuild:
                        rebuild[uid] = set()
                    rebuild[uid].add(speedType)

    for speedType, uids in sessionBests.items():
        if speedType not in rankingBests:
            if verbose:
                print('Warning: {} not in EventRankings'.format(speedType))
        else:
            for uid, speed in uids.items():
                if uid not in rankingBests[speedType]:
                    if speed > 0:
                        if verbose:
                            print('{} ({}) {} not in EventRankings - {:.2f} session'.format(
                                users[uid], uid, speedType, speed))
                        if uid not in rebuild:
                            rebuild[uid] = set()
                        rebuild[uid].add(speedType)
                else:
                    if rankingBests[speedType][uid] != speed:
                        if verbose:
                            print('{} ({}) mismatch for {} - {:.2f} ranking vs {:.2f} session'.format(
                                users[uid], uid, speedType, rankingBests[speedType][uid] / 1.852, speed / 1.852))
                        if uid not in rebuild:
                            rebuild[uid] = set()
                        rebuild[uid].add(speedType)

    if verbose and rebuild:
        print()

    if rebuild:
        print('{} / {} users need a rebuild in {} ({}):'.format(len(rebuild), len(users), events[eventId], eventId))
        for uid in rebuild:
            print('{} ({}) - {}'.format(users[uid], uid, ', '.join(rebuild[uid])))
    else:
        print('No users need a rebuild in {} ({})'.format(events[eventId], eventId))

    print()

In [9]:
if __name__ == '__main__':
    projdir = os.path.realpath(os.path.join(sys.path[0], '..'))
    verbose = False

    basedir = os.path.join(projdir, 'cache', 'EventRankings')
    
    eventIds = [int(dirname) for dirname in os.listdir(basedir)]
    
    for eventId in eventIds:
        if verbose:
            print('Checking event {}'.format(eventId))

        eventDir = os.path.join(basedir, str(eventId))
        
        rankings = loadEventRankings(eventDir)
        rankingBests, rankingDuplicates = parseEventRankings(rankings, verbose=verbose)

        try:
            sessions = loadEventSessions(eventId)
            sessionBests = parseEventSessions(sessions)

            compareBests(eventId, rankingBests, rankingDuplicates, sessionBests, verbose=verbose)
            
        except FileNotFoundError:
            # I have intentionally cached the event rankings (but not sessions) for the British Foil Speed Challenge 2022
            pass
        
    print('All done!')

No users need a rebuild in 2022 FFF SPEED CHALLENGE (727)

3 / 22 users need a rebuild in 2022 Funsport Makkum GPS Speed Challenge (740):
Paul Salomons (3078) - speed_10sec, speed_500, speed_2sec, speed_hour, speed_avg, speed_250, speed_100, speed_mile
Rob de Jong (4307) - speed_10sec, speed_500, speed_2sec, speed_hour, speed_avg, speed_250, speed_100
Jan Hendrik de Bruin (2574) - speed_10sec, speed_500, speed_2sec, speed_hour, speed_avg, speed_250, speed_100

1 / 29 users need a rebuild in 2022 Irish Speed Masters (717):
Gildas Bechet (3671) - speed_10sec, speed_500, speed_2sec, speed_hour, speed_avg, speed_250, speed_100

No users need a rebuild in Surf like a dude Zuidlaardermeer Speed Challenge 2022 (730)

No users need a rebuild in Norwegian speedmaster - 2022 (719)

6 / 86 users need a rebuild in SSS Go big or go Home 2021-2022 (715):
Hans Prins (5376) - speed_10sec, speed_500, speed_2sec, speed_hour, speed_avg, speed_250, speed_100, speed_mile
Sven Hulst (23299) - speed_10sec, s